In [2]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.8 MB/s eta 0:00:00


In [3]:
!pip install flask pyngrok pycryptodome

In [ ]:
from flask import Flask, request, send_file, render_template_string
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from io import BytesIO
from pyngrok import conf, ngrok
import hashlib

app = Flask(__name__)
conf.get_default().auth_token = "2ujCdDkaOVBkBmrqIRJACyVCwRD_2nF1MMtFrfkdkjTpKxkJT"

# ---------------------------
# AES với khóa băm SHA-256
# ---------------------------
BLOCK_SIZE = 16  # AES block size

def normalize_key(key: str) -> bytes:
    return hashlib.sha256(key.encode()).digest()  # 32 bytes (AES-256)

def encrypt_aes(data: bytes, key: str) -> bytes:
    cipher = AES.new(normalize_key(key), AES.MODE_ECB)
    return cipher.encrypt(pad(data, BLOCK_SIZE))

def decrypt_aes(data: bytes, key: str) -> bytes:
    cipher = AES.new(normalize_key(key), AES.MODE_ECB)
    return unpad(cipher.decrypt(data), BLOCK_SIZE)

# ---------------------------
# HTML giao diện web
# ---------------------------
HTML = '''
<!DOCTYPE html>
<html lang="vi">
<head>
    <meta charset="UTF-8">
    <title>Mã hóa/Giải mã file AES</title>
    <style>
        body {
            font-family: 'Segoe UI', sans-serif;
            background: linear-gradient(to right, #232526, #ce1313);
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
        }
        .container {
            background: #ffffffcc;
            backdrop-filter: blur(5px);
            padding: 30px 40px;
            border-radius: 16px;
            box-shadow: 0 8px 24px rgba(0, 0, 0, 0.2);
            width: 400px;
        }
        h2 {
            text-align: center;
            margin-bottom: 25px;
            color: #333;
        }
        p {
            margin: 10px 0;
            font-weight: bold;
        }
        input[type="file"], input[type="text"], select, button {
            width: 100%;
            padding: 10px 12px;
            margin-top: 6px;
            border-radius: 8px;
            border: 1px solid #ccc;
            box-sizing: border-box;
            font-size: 14px;
        }
        button {
            background: linear-gradient(to right, #232526, #ce1313);
            color: white;
            font-weight: bold;
            border: none;
            cursor: pointer;
            margin-top: 20px;
            transition: background 0.3s ease;
        }
        button:hover {
            background: linear-gradient(to right, #5b86e5, #36d1dc);
        }
        .error {
            color: red;
            font-weight: bold;
            margin-top: 15px;
            text-align: center;
        }
    </style>
</head>
<body>
    <div class="container">
        <h2>Mã hóa / Giải mã file AES</h2>
        <form method="POST" enctype="multipart/form-data">
            <p>Chọn file:
            <input type="file" name="file" required></p>

            <p>Nhập khóa (tự do):
            <input type="text" name="key" required></p>

            <p>Chế độ:
            <select name="mode" required>
                <option value="encrypt">Mã hóa</option>
                <option value="decrypt">Giải mã</option>
            </select></p>

            <button type="submit">Thực hiện</button>
        </form>
        {% if error %}
            <div class="error">{{ error }}</div>
        {% endif %}
    </div>
</body>
</html>
'''

@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        f = request.files['file']
        key = request.form['key']
        mode = request.form['mode']
        data = f.read()

        try:
            if mode == 'encrypt':
                result = encrypt_aes(data, key)
                filename = "da_ma_hoa.aes"
            else:
                result = decrypt_aes(data, key)
                filename = "da_giai_ma.txt"

            buffer = BytesIO(result)
            buffer.seek(0)
            return send_file(buffer, as_attachment=True, download_name=filename)

        except Exception as e:
            return render_template_string(HTML, error=str(e))

    return render_template_string(HTML, error=None)

# Khởi tạo ngrok
public_url = ngrok.connect(5000)
print(f"📌 Truy cập web tại: {public_url}")

app.run(port=5000, use_reloader=False)


📌 Truy cập web tại: NgrokTunnel: "https://3730-34-71-26-240.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [22/May/2025 10:19:27] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/May/2025 10:19:27] "GET /favicon.ico HTTP/1.1" 404 -
